# Scrape the [Texas State Chemist Active Fertilizer Registry](https://otscweb.tamu.edu/Reports/ActiveFertRegistrant.aspx)

In [1]:
import bs4
import pandas as pd
import requests

## Locate the funds table on the page

In [2]:
tx_state_chemist_page = requests.get('https://otscweb.tamu.edu/Reports/ActiveFertRegistrant.aspx')

In [3]:
soup = bs4.BeautifulSoup(tx_state_chemist_page.text, 'html.parser')

In [4]:
table = soup.find('table')

In [5]:
rows = table.find_all('tr')

Count up the rows

In [6]:
len(rows)

1223

# Extract the data from the company table

In [7]:
columns = ['permit','name','address']

fertilizer_registrants = pd.DataFrame(columns=columns) # Create empty dataframe

for item in rows[1:]: # Skip the column headers
    # Iterate through the table rows
    cells = item.find_all('td')
    permit = cells[0].text
    name = cells[1].text
    address = ' '.join([value for value in cells[2].find('span').contents if type(value) is bs4.element.NavigableString]).replace('\n','')
    fertilizer_registrants = pd.concat([fertilizer_registrants,pd.DataFrame([[permit,name,address]],columns=columns)])

fertilizer_registrants = fertilizer_registrants.set_index('permit')
    
fertilizer_registrants.head(10)

,name,address
permit,,
203962,"2K AGRI SERVICES, INC.","201 HWY 62/82 LORENZO, TX 79343"
705463,3 TIER TECHNOLOGIES LLC,"250 NATIONAL PLACE #142 LONGWOOD, FL 32750"
203926,"3D AG SERVICES, INC.","P. O. BOX 337 RISING STAR, TX 76471"
706920,3H TRADING COMPANY,8505 TECHNOLOGY FOREST PLACE STE 1102 THE WOOD...
707303,4C FEED & FERTILIZER,"P. O. BOX 632564 NACOGDOCHES, TX 75963"
705019,"5-J FEED & FARM SUPPLY, INC.","P. O. BOX 151 LEROY, TX 76654"
708599,A & M FARM SUPPLY,"2100 E. LOOP 281 LONGVIEW, TX 75605"
709126,A DIVISION OF OROGRO,"30 N. GOULD ST., SUITE R SHERIDAN, WY 82801"
203732,"A-MARC PRODUCTS, INC.","506 WEST 20TH ST. MULESHOE, TX 79347"


## Check the extracted data

In [8]:
fertilizer_registrants.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1222 entries, 203962 to 203833
Data columns (total 2 columns):
name       1222 non-null object
address    1222 non-null object
dtypes: object(2)
memory usage: 14.3+ KB


In [9]:
fertilizer_registrants.shape

(1222, 2)

Check for nulls

In [10]:
nulls_scrape = fertilizer_registrants[fertilizer_registrants.isna().any(axis=1)]
nulls_scrape.head()

,name,address
permit,,


## Export scraped data 

In [11]:
fertilizer_registrants.to_csv('tx_fertilizer_registrants200114.csv')

Check the .csv we just created

In [12]:
check_csv = pd.read_csv('tx_fertilizer_registrants200114.csv')

Check for nulls in csv

In [13]:
nulls_csv = fertilizer_registrants[fertilizer_registrants.isna().any(axis=1)]
nulls_csv.head()

,name,address
permit,,
